In [4]:
import pandas as pd
import numpy as np
import glob

In [13]:
##au represent fluorescence intensity
##dc represent diffusion coefficient


def analyze(f):
    df=pd.read_csv(f,encoding = 'shift-jis', dtype={'No':'object',  'Time[frame]':'int','x[px]':'float','y[px]':'float','sigmaX[px]':'float','sigmaY[px]':'float',
                          'I[au]':'float'},header = 0, usecols=[0,1,2,3,4,5,6,10])

    df= df.rename(columns={'x[px]':'X Position(pixels)','y[px]':'Y Position(pixels)'})

    x = df['No']
    df['Index'] = x.groupby((x != x.shift()).cumsum()).cumcount()

    df= df.reset_index(drop=True)

    df["No"].values[0] = 0

    for i in range(len(df.index)-1):
        if df["No"].values[i+1] != df["No"].values[i]:
            if df["Index"].values[i+1] < df["Index"].values[i]:
                df["No"].values[i+1] = df["No"].values[i] + 1
            else:
                df["No"].values[i+1] = df["No"].values[i]

    df["au"]=2*3.14*df["sigmaX[px]"]*df["sigmaY[px]"]*df["I[au]"]
    df["X Position(um)"] = df["X Position(pixels)"] *6.5/90  
    df["Y Position(um)"] = df["Y Position(pixels)"] *6.5/90
    df["dX"] = df["X Position(um)"].diff()
    df.loc[df["Index"] == 0, "dX"] = np.nan
    df["dY"] = df["Y Position(um)"].diff()
    df.loc[df["Index"] == 0, "dY"] = np.nan
    df["dc"] = (df["dX"] ** 2 + df["dY"] ** 2)/(4*0.1/3)

    au=[]
    dc=[]
    for i in range(len(df)):
        au.append(df["au"].values[i])
        dc.append(df["dc"].values[i])
    
    return au,dc
